In [ ]:
from toolkit.lap.track import *
from toolkit.lap.channels import *
from toolkit.lap.gps import *
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import rerun as rr


In [ ]:
raw_track = load_track_from_mat('./data/lap_data/2017_Michigan_Endurance_First4Laps.mat')
data_dict = parse_car_data_mat(raw_track)
track_gps = gps_from_channels(data_dict["GPS_Latitude"], data_dict["GPS_Longitude"], data_dict["GPS_Altitude"], data_dict["GPS_Time"])

In [ ]:

track = Track(track_gps, data_dict, 50, 0.95)

In [ ]:
def display_track(a_track):
    rr.init("log viewer", spawn=True)
    for name in WELL_KNOWN_KEYS:
        time, data = a_track.get_channel(name)
        rr.send_columns(
            name,
            times=[rr.TimeSecondsColumn("step", time)],
            components=[rr.components.ScalarBatch(data)]
        )
    rr.log("track", rr.LineStrips2D(a_track.smooth_gps.get_2d_track(enu=True), radii=0.002, colors=[0, 255, 255, 255]), timeless=True)
    rr.send_columns(
        "gps",
        times=[rr.TimeSecondsColumn("step", a_track.gps.time)],
        components=[rr.Points2D.indicator(),
        rr.components.Position2DBatch(a_track.gps.get_2d_track(enu=True)),]
    )


In [ ]:
display_track(track)

In [ ]:
gps_track = track.smooth_gps.get_2d_track()
[(gps_track[i], gps_track[i + 1]) for i in range(len(gps_track) - 1)]

In [ ]:
np.arange(track.gps.time.shape[0])

In [ ]:
track.gps.time[-60:]

In [ ]:
data_dict.keys()